In [ ]:
using GeoDataFrames
using DataFrames
using Plots
using Shapefile
using Random
using ArchGDAL
using GeoInterface

In [ ]:
###Load shapefile

shapefile_path = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\MT_Municipios_2022\\MT_Municipios_2022.shp"
municipalities = Shapefile.Table(shapefile_path) |> DataFrame

In [ ]:
# Fi
municipios_filtrados = [
    "Cáceres", "Primavera do Leste", "Rondonópolis", "Sinop", "Sorriso", 
    "Guarantã do Norte", "Matupá", "Peixoto de Azevedo", "Terra Nova do Norte", 
    "Nova Santa Helena", "Itaúba", "Lucas do Rio Verde", "Nova Mutum",
    "Barra do Garças", "Campo Verde"
]

# Filtrando os dados
filtered_municipalities = filter(row -> row.NM_MUN in municipios_filtrados, municipalities)

# Exibindo os dados filtrados
println(filtered_municipalities)

In [ ]:
# Caminho para o shapefile das rodovias
shapefile_path_rodovias = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\Rodovias\\SNV_202410A.shp"
# Carregar o shapefile das rodovias
rodovias = Shapefile.Table(shapefile_path_rodovias) |> DataFrame

# Filtrar as rodovias com a coluna vl_br igual a 070 ou 163 e sg_uf igual a MT
filtered_rodovias_070 = filter(row -> row.vl_br == "070" && row.sg_uf == "MT", rodovias)
filtered_rodovias_163 = filter(row -> row.vl_br == "163" && row.sg_uf == "MT", rodovias)

In [ ]:
# Combinar os dataframes filtrados
filtered_highways_MT = vcat(filtered_rodovias_070, filtered_rodovias_163)

# Agrupar por vl_br
grouped_highways = groupby(filtered_highways_MT, :vl_br)

In [ ]:
#Combine geometries
function combine_geometries(geometries)

    return mapfoldl(
        x -> GeoInterface.convert(ArchGDAL, x), # map - apply to each element
        ArchGDAL.union, # reduce 2 elements to 1 element
        geometries # the vector to perform the operation on
    )
end

In [ ]:
# Applying function
combined_highways = combine(grouped_highways, :geometry => combine_geometries => :geometry)

#Showing results
println(combined_highways)

In [ ]:
# Gerar cores aleatórias para os municípios filtrados
colors = [RGB(rand(), rand(), rand()) for _ in 1:length(municipios_filtrados)]

In [ ]:
# Definir cores para combined_geometries
combined_colors = [:black, :gray]  # Adicione mais cores conforme necessário

In [ ]:
# Plotar todos os municípios e as rodovias filtradas
plot(size=(1000, 800))  # Definir tamanho do gráfico
plot!(legend=:outerright)
plot!(municipalities.geometry, color=:white, linecolor=:black, label=false)

for (i, municipio) in enumerate(filtered_municipalities.geometry)
    plot!(municipio, color=colors[i], label=municipios_filtrados[i])
end

for (i, geometry) in enumerate(combined_highways.geometry)
    plot!(geometry, color=combined_colors[i], linewidth=4, label="BR-" * string(combined_highways.vl_br[i]))
end

xlabel!("")
ylabel!("")
title!("")
plot!(legend=:outerright, legendfontsize=8)  # Colocar a legenda ao lado e diminuir o tamanho da legenda
plot!(framestyle=:none, grid=false)  # Excluir eixos e marcas de quadrado

In [ ]:
# Salvar o gráfico
savefig("C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Dissertação\\Quadros e gráficos\\Graph-Cities BR070 and BR163.png")
